## Autoencoder
Refs: [DL book](https://www.deeplearningbook.org/contents/autoencoders.html); [inspired by](https://gist.github.com/AFAgarap/4f8a8d8edf352271fa06d85ba0361f26).


In [79]:
import torch       
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from sklearn.decomposition import PCA
from scipy.stats import special_ortho_group
import numpy as np

np.set_printoptions(precision=3, suppress=True)
torch.set_printoptions(precision=3, sci_mode=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [81]:
x = special_ortho_group.rvs(2)
z = torch.randn(2)
print( x, z, (z @ x).float())
v, _  = gen_data(**{**params, "true_dim":2})
print(v)
print(data0)

[[-0.693 -0.721]
 [ 0.721 -0.693]] tensor([0.442, 1.298]) tensor([ 0.629, -1.218])
tensor([[ 0.195, -0.151, -0.151,  ...,  0.066,  0.088,  0.031],
        [-1.100,  1.286,  0.979,  ..., -0.452, -1.255,  0.434],
        [-0.335, -0.075,  0.163,  ..., -0.051,  0.438, -0.525],
        ...,
        [-0.364,  0.319,  0.293,  ..., -0.130, -0.227, -0.007],
        [ 0.278, -0.574, -0.320,  ...,  0.161,  0.754, -0.460],
        [ 1.233, -0.439, -0.808,  ...,  0.320, -0.353,  0.925]])
tensor([[ 0.539, -0.808,  0.757,  ..., -0.521, -1.217, -1.289],
        [-0.663,  0.374, -1.268,  ...,  0.784,  0.302, -0.795],
        [ 1.319, -1.233, -0.917,  ...,  0.871, -0.016, -0.044],
        ...,
        [ 0.801,  0.157,  0.312,  ...,  0.545, -0.049, -0.144],
        [ 1.947, -0.444,  1.982,  ..., -0.452, -0.150,  1.633],
        [ 0.113,  0.215,  0.956,  ..., -0.076,  0.047, -0.731]])


In [82]:
# architecture
class shallow_AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_layer = nn.Linear(
            in_features=kwargs["input_width"], out_features=kwargs["hidden_width"], bias=kwargs["bias"]
        )
        self.decoder_layer = nn.Linear(
            in_features=kwargs["hidden_width"], out_features=kwargs["input_width"], bias=kwargs["bias"]
        )
        
        if 'linear' in kwargs:
            self.linear = kwargs['linear']
        else:
            self.linear = False

    def forward(self, features):
        activation = self.encoder_layer(features)
        if not self.linear:
            activation = F.relu(activation)
        reconstructed = self.decoder_layer(activation)
        return reconstructed
    
def train(model, epochs, train_loader, optimizer, criterion, verbose=True):
    for epoch in range(epochs):
        loss = 0
        for batch_features in train_loader:
            batch_features = batch_features.to(device)
            optimizer.zero_grad()

            outputs = model(batch_features)
            train_loss = criterion(outputs, batch_features)
            train_loss.backward()
            optimizer.step()
            loss += train_loss.item()

        loss = loss / len(train_loader)
        if verbose:
            print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))
    if verbose:
        print("===================\n")
    return model, loss

In [85]:
def gen_data(**kwargs):
    data0 = torch.randn(kwargs['samples'], kwargs['input_width'])
    
    if ('true_dim' in kwargs) and (kwargs['true_dim']<kwargs['input_width']):
        data0[:,kwargs['true_dim']:] = 0
        data0 = data0 @ special_ortho_group.rvs(kwargs['input_width'])
        data0 = data0.float()
        
    train_loader = torch.utils.data.DataLoader(
        data0, batch_size=kwargs['batch_size']
    )
    
    return data0, train_loader

def PCA_compare(model, model_loss, data0, **kwargs):
    U, s, V = np.linalg.svd(data0, full_matrices=False)

    b=np.append(s[:kwargs['hidden_width']], np.zeros(len(s)-kwargs['hidden_width']))
    L_opt = np.sum(s**2-b**2)*kwargs['batch_size']/kwargs['samples']

    print("Model loss = ", model_loss)
    print("PCA loss = ", L_opt)

#     print("\nModel params:")
#     for n, p in model.named_parameters():
#         print(n, p)

    with torch.no_grad():
        print("model estimate:\n", model(data0.to(device)).detach().cpu().numpy())
        print("\nPCA estimate:\n", U  @ np.diag(b) @ V)

## Linear activation implies that AutoEncoder = PCA

### Underparametrization: hidden_width < true_dim

In [90]:
# hyperparams
params = {
    # data    
    "input_width": 10, 
    "samples": 10000,
    "true_dim":5,
    
    # model
    "hidden_width":3,
    "linear":True,
    "bias": False,
    
    # training
    "lr": 1e-3,
    "batch_size": 100,
    "epochs": 100
}

data0, train_loader = gen_data(**params)

model = shallow_AE(**params).to(device)
opt = optim.SGD(model.parameters(), lr=params['lr'])
crit = nn.MSELoss(reduction='sum')

model_trained, model_loss = train(model, params["epochs"], train_loader, opt, crit, verbose=False)

PCA_compare(model_trained, model_loss, data0, **params)

Model loss =  192.28164459228515
PCA loss =  191.9860397593492
model estimate:
 [[ 0.216 -0.64   0.301 ... -1.359  0.21   0.481]
 [-0.151 -0.795  1.515 ... -0.082 -0.028  0.253]
 [-0.549 -0.317 -1.435 ... -0.015 -0.863  0.082]
 ...
 [-0.211  0.336  0.712 ...  1.422 -0.085 -0.402]
 [ 0.096  0.005 -1.608 ... -1.101 -0.139  0.229]
 [-0.233 -1.06   0.029 ... -1.133 -0.376  0.547]]

PCA estimate:
 [[ 0.217 -0.672  0.312 ... -1.35   0.198  0.482]
 [-0.134 -0.807  1.513 ... -0.068 -0.035  0.257]
 [-0.558 -0.319 -1.431 ... -0.033 -0.871  0.098]
 ...
 [-0.206  0.347  0.709 ...  1.41  -0.079 -0.396]
 [ 0.092  0.029 -1.619 ... -1.092 -0.131  0.216]
 [-0.222 -1.081  0.036 ... -1.134 -0.384  0.558]]


#### Is ReLU better than linear network = PCA?

In [91]:
params["linear"] = False


model = shallow_AE(**params).to(device)
opt = optim.Adam(model.parameters(), lr=params['lr'])
crit = nn.MSELoss(reduction='sum')

model_trained, model_loss = train(model, params["epochs"], train_loader, opt, crit, verbose=True)

PCA_compare(model_trained, model_loss, data0, **params)

epoch : 1/100, loss = 504.310846
epoch : 2/100, loss = 457.270787
epoch : 3/100, loss = 427.894512
epoch : 4/100, loss = 402.553207
epoch : 5/100, loss = 374.410618
epoch : 6/100, loss = 349.275901
epoch : 7/100, loss = 333.778637
epoch : 8/100, loss = 325.870995
epoch : 9/100, loss = 322.070581
epoch : 10/100, loss = 320.252884
epoch : 11/100, loss = 319.393210
epoch : 12/100, loss = 318.987177
epoch : 13/100, loss = 318.778345
epoch : 14/100, loss = 318.651924
epoch : 15/100, loss = 318.553513
epoch : 16/100, loss = 318.476890
epoch : 17/100, loss = 318.409689
epoch : 18/100, loss = 318.342097
epoch : 19/100, loss = 318.267109
epoch : 20/100, loss = 318.195985
epoch : 21/100, loss = 318.118884
epoch : 22/100, loss = 318.045275
epoch : 23/100, loss = 317.968670
epoch : 24/100, loss = 317.895292
epoch : 25/100, loss = 317.821827
epoch : 26/100, loss = 317.754227
epoch : 27/100, loss = 317.685171
epoch : 28/100, loss = 317.613226
epoch : 29/100, loss = 317.545508
epoch : 30/100, loss = 

### Overparametrization: hidden_width > true_dim

In [93]:
params['true_dim'] = params['hidden_width']-1
params['linear'] = True

data0, train_loader = gen_data(**params)

model = shallow_AE(**params).to(device)
opt = optim.SGD(model.parameters(), lr=params['lr'])
crit = nn.MSELoss(reduction='sum')

model_trained, model_loss = train(model, params["epochs"], train_loader, opt, crit, verbose=False)

model_loss
PCA_compare(model_trained, model_loss, data0, **params)

Model loss =  3.606985374444982e-12
PCA loss =  -5.792436355976684e-06
model estimate:
 [[-0.581 -0.242 -0.506 ... -0.126  0.448 -0.004]
 [-0.248  0.156  0.158 ...  0.11   0.102 -0.025]
 [ 0.076 -0.252 -0.344 ... -0.164  0.039  0.026]
 ...
 [-0.235 -0.476 -0.751 ... -0.291  0.312  0.032]
 [-0.484  0.561  0.681 ...  0.379  0.11  -0.07 ]
 [-0.104 -0.073 -0.134 ... -0.041  0.091  0.002]]

PCA estimate:
 [[-0.581 -0.242 -0.506 ... -0.126  0.448 -0.004]
 [-0.248  0.156  0.158 ...  0.11   0.102 -0.025]
 [ 0.076 -0.252 -0.344 ... -0.164  0.039  0.026]
 ...
 [-0.235 -0.476 -0.751 ... -0.291  0.312  0.032]
 [-0.484  0.561  0.681 ...  0.379  0.11  -0.07 ]
 [-0.104 -0.073 -0.134 ... -0.041  0.091  0.002]]


In [94]:
params['linear'] = False

data0, train_loader = gen_data(**params)

model = shallow_AE(**params).to(device)
opt = optim.SGD(model.parameters(), lr=params['lr'])
crit = nn.MSELoss(reduction='sum')

model_trained, model_loss = train(model, params["epochs"], train_loader, opt, crit, verbose=False)

model_loss
PCA_compare(model_trained, model_loss, data0, **params)

Model loss =  31.719064273834228
PCA loss =  -4.535846459888372e-06
model estimate:
 [[-0.242 -0.687 -1.209 ... -0.358 -0.776  0.467]
 [ 0.472  1.358  0.309 ...  0.394 -0.017  0.809]
 [ 0.09   0.263 -0.352 ...  0.014 -0.31   0.499]
 ...
 [-0.277 -0.785 -1.381 ... -0.409 -0.886  0.533]
 [ 0.326  0.939  0.214 ...  0.272 -0.012  0.559]
 [ 0.003  0.005  0.392 ...  0.061  0.291 -0.322]]

PCA estimate:
 [[-0.092 -0.254 -1.217 ... -0.249 -0.861  0.814]
 [ 0.425  1.22   0.442 ...  0.379  0.107  0.589]
 [ 0.117  0.344 -0.641 ... -0.008 -0.54   0.803]
 ...
 [-0.397 -1.133 -1.078 ... -0.452 -0.596  0.008]
 [ 0.271  0.783 -0.065 ...  0.19  -0.192  0.669]
 [ 0.095  0.266  0.681 ...  0.171  0.459 -0.358]]
